# Solving with python


In [1]:
import math
from typing import Callable, List, Tuple

try:
    import matplotlib.pyplot as plt
    HAVE_MPL = True
except Exception:
    HAVE_MPL = False


_SQRT2 = math.sqrt(2.0)

def norm_cdf(x: float) -> float:
    
    return 0.5 * (1.0 + math.erf(x / _SQRT2))

In [2]:
def bs_call(S: float, K: float, T: float, vol: float, r: float, q: float = 0.0) -> float:

    if T <= 0.0:
        return max(S - K, 0.0)

    if vol <= 0.0:
        return max(S * math.exp(-q * T) - K * math.exp(-r * T), 0.0)

    sqrtT = math.sqrt(T)
    d1 = (math.log(S / K) + (r - q + 0.5 * vol * vol) * T) / (vol * sqrtT)
    d2 = d1 - vol * sqrtT
    return S * math.exp(-q * T) * norm_cdf(d1) - K * math.exp(-r * T) * norm_cdf(d2)


def implied_vol_call_bisection(
    C_target: float,
    S: float,
    K: float,
    T: float,
    r: float,
    q: float = 0.0,
    vol_lo: float = 1e-8,
    vol_hi: float = 5.0,
    tol: float = 1e-10,
    max_iter: int = 200
) -> float:
    
    C_min = max(0.0, S * math.exp(-q * T) - K * math.exp(-r * T))
    C_max = S * math.exp(-q * T)
    C = min(max(C_target, C_min), C_max)

    
    if abs(C - C_min) < 1e-14:
        return vol_lo
    if abs(C - C_max) < 1e-14:
        return vol_hi

    def f(sig: float) -> float:
        return bs_call(S, K, T, sig, r, q) - C

    f_lo = f(vol_lo)
    f_hi = f(vol_hi)

    
    if f_lo > 0.0:
        return vol_lo

    
    expand_tries = 0
    while f_hi < 0.0 and vol_hi < 10.0 and expand_tries < 10:
        vol_hi *= 1.5
        f_hi = f(vol_hi)
        expand_tries += 1
    if f_hi < 0.0:
    
        return vol_hi

    
    lo, hi = vol_lo, vol_hi
    for _ in range(max_iter):
        mid = 0.5 * (lo + hi)
        f_mid = f(mid)
        if f_mid > 0.0:
            hi = mid
        else:
            lo = mid
        if hi - lo < tol * (1.0 + mid):
            return 0.5 * (lo + hi)
    return 0.5 * (lo + hi)


In [3]:
def call_jumpdiffusion_mixture(
    S0: float,
    K: float,
    T: float,
    sigma: float,
    r: float,
    lamQ: float,
    gamma: float,
    eps_tail: float = 1e-12
) -> float:
    
    if T <= 0.0:
        return max(S0 - K, 0.0)

    lamT = lamQ * T
    q_eff = - lamQ * gamma

    w = math.exp(-lamT)
    price = w * bs_call(S0, K, T, sigma, r, q_eff)
    sum_w = w
    j = 0

    while (1.0 - sum_w) > eps_tail:
        j += 1
        w = w * lamT / j
        Sj = S0 * (1.0 + gamma) ** j
        price += w * bs_call(Sj, K, T, sigma, r, q_eff)
        sum_w += w

        if j > 1000:
            break

    return price

In [4]:
def batch_iv_surface(
    S0: float,
    r: float,
    sigma: float,
    lamQ: float,
    gamma: float,
    maturities: List[float],
    moneyness: List[float],
    make_plots: bool = True
) -> List[Tuple[float, List[Tuple[float, float]]]]:

    results = []
    for T in maturities:
        curve = []
        for m in moneyness:
            K = S0 * m

            C = call_jumpdiffusion_mixture(S0, K, T, sigma, r, lamQ, gamma)

            iv = implied_vol_call_bisection(C, S0, K, T, r, q=0.0)
            curve.append((m, iv))
        results.append((T, curve))

        if make_plots and HAVE_MPL:
            xs = [m for (m, _) in curve]
            ys = [iv for (_, iv) in curve]
            plt.figure()
            plt.plot(xs, ys, marker='o')
            plt.xlabel('K / S0')
            plt.ylabel('Implied volatility')
            plt.title(f'Implied vol vs K/S0 (T = {T:.2f})')
            plt.grid(True)
            plt.savefig(f'iv_T_{T:.2f}.pdf', bbox_inches='tight')
            plt.close()

    return results

In [5]:
import seaborn as sns
sns.set_theme()

S0    = 100.0
r     = 0.04
sigma = 0.20
lamQ  = 0.20
gamma = -0.08 

maturities = [0.02, 0.08, 0.25, 0.50]     
moneyness  = [0.8, 0.9, 1.0, 1.1]

iv_grid = batch_iv_surface(S0, r, sigma, lamQ, gamma, maturities, moneyness, make_plots=True)

for T, curve in iv_grid:
    print(f"\nT = {T:.2f}")
    for m, iv in curve:
        print(f"  K/S0={m:>4.2f}  IV={iv:.6f}")
if not HAVE_MPL:
    print("\n(matplotlib not installed; plots were skipped)")



T = 0.02
  K/S0=0.80  IV=0.000000
  K/S0=0.90  IV=0.000000
  K/S0=1.00  IV=0.196264
  K/S0=1.10  IV=0.199314

T = 0.08
  K/S0=0.80  IV=0.000000
  K/S0=0.90  IV=0.000000
  K/S0=1.00  IV=0.190916
  K/S0=1.10  IV=0.197153

T = 0.25
  K/S0=0.80  IV=0.000000
  K/S0=0.90  IV=0.000000
  K/S0=1.00  IV=0.181181
  K/S0=1.10  IV=0.191174

T = 0.50
  K/S0=0.80  IV=0.000000
  K/S0=0.90  IV=0.106855
  K/S0=1.00  IV=0.170871
  K/S0=1.10  IV=0.183544
